In [2]:
# Import libraries
import os

import numpy as np
import pandas as pd

from statsmodels.tsa.statespace.sarimax import SARIMAX
# from keras.models import Sequential
# from keras.layers import LSTM, Dense

In [3]:
df_finbert = pd.read_csv(os.path.join('..', 'data','RedditNews_FinBert.csv'))
df_vader = pd.read_csv(os.path.join('..', 'data','RedditNews_Vader.csv'))

df_stock = pd.read_csv(os.path.join('..', 'data','upload_DJIA_table_preprocessed.csv'))

In [4]:
# Calculate the mean sentiment score for each stock for each day
df_finbert = df_finbert.groupby('Date') \
       .agg({'sentiment_FinBert_positive':'mean',
            'sentiment_FinBert_negative':'mean',
            'sentiment_FinBert_neutral':'mean'}) \
       .rename(columns={'sentiment_FinBert_positive':'sentiment_FinBert_positive',
                        'sentiment_FinBert_negative':'sentiment_FinBert_negative',
                        'sentiment_FinBert_neutral':'sentiment_FinBert_neutral'}) \
       .reset_index()

# Calculate the mean sentiment score for each stock for each day
df_vader = df_vader.groupby('Date') \
       .agg({'sentiment_vader':'mean'}) \
       .rename(columns={'sentiment_vader':'sentiment_vader'}) \
       .reset_index()

In [5]:
df_finbert = df_finbert.merge(df_stock[['Date', 'returns']], on='Date')
df_vader = df_vader.merge(df_stock[['Date', 'returns']], on='Date')

df_finbert['Date'] = pd.to_datetime(df_finbert['Date'])
df_vader['Date'] = pd.to_datetime(df_vader['Date'])

# set the index
df_finbert.set_index('Date', inplace=True)
df_vader.set_index('Date', inplace=True)

In [7]:
# Define the ARIMAX model
# For this example, let's use the order (1,1,1) for ARIMA, but you'll need to determine the best order for your data
model = SARIMAX(df_vader['returns'], exog=df_vader['sentiment_vader'], order=(1,1,1))

# Fit the model
results = model.fit(disp=-1)

C:\Users\duzzi\anaconda3\envs\MasterThesis\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\duzzi\anaconda3\envs\MasterThesis\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\duzzi\anaconda3\envs\MasterThesis\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [10]:
df_vader['predicted_returns'] = results.predict(start=1, end=len(df_vader), exog=df_vader['sentiment_vader'])
df_vader['residuals'] = df_vader['returns'] - df_vader['predicted_returns']

C:\Users\duzzi\anaconda3\envs\MasterThesis\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\duzzi\anaconda3\envs\MasterThesis\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


ValueError: Provided exogenous values are not of the appropriate shape. Required (1, 1), got (1988,).

In [26]:
# Convert them into pandas dataframes
y_pred_finbert_df = pd.DataFrame(y_pred_finbert, columns=['y_pred_FinBert'])
y_test_finbert_df = pd.DataFrame(y_test_finbert, columns=['y_test_FinBert'])
y_pred_vader_df = pd.DataFrame(y_pred_vader, columns=['y_pred_Vader'])
y_test_vader_df = pd.DataFrame(y_test_vader, columns=['y_test_Vader'])

# Combine them into a single dataframe
results = pd.concat([y_pred_finbert_df, y_test_finbert_df,y_pred_vader_df, y_test_vader_df], axis=1)

In [28]:
results
results.to_csv('RedditNews_Results.csv', index=False)